In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from model.scenarios import (
    globalParams,
    params,
    strategies,
    scenario_defs,
    apply_scenario,
)

from model.engine import (
    analyze_status_quo,
    run_all_strategies,
)


In [ ]:
# 1. Load data from CSV


income = pd.read_csv("data/income_statement_2017_2022.csv")
stud_hist = pd.read_csv("data/student_weeks_history.csv")


# 2. Economic calibration


# 2.1 Price per student
income["PriceStandardPerWeek"] = (
    income["IncomeStandard"] / income["StudentWeeksStandard"]
)

with np.errstate(divide="ignore", invalid="ignore"):
    income["PriceWeekendPerStudent"] = (
        income["IncomeWeekend"] / income["StudentWeeksWeekend"]
    )
    income.loc[
        ~np.isfinite(income["PriceWeekendPerStudent"]),
        "PriceWeekendPerStudent"
    ] = np.nan

# Base: average over 2019–2022 (more stable period)
mask_recent = income["Year"] >= 2019
price_std_base = income.loc[mask_recent, "PriceStandardPerWeek"].mean()
price_wkd_base = income.loc[mask_recent, "PriceWeekendPerStudent"].mean()

# 2.2 Cost ratios
income["OperatingCashCosts"] = (
    income["CostOfSales"]
    + income["AdminStaffSalaries"]
    + income["MarketingAdvertising"]
    + income["PropertyCosts"]
    + income["OverheadAdminExpenses"]
)

income["RoyaltyRate"] = income["Royalties"] / income["TotalIncome"]
income["OperatingCashRate"] = income["OperatingCashCosts"] / income["TotalIncome"]
income["OtherOperatingRate"] = income["OperatingCashRate"] - income["RoyaltyRate"]

# "Normal" years: 2017, 2018, 2019 and 2022
mask_normal = income["Year"].isin([2017, 2018, 2019, 2022])

royalty_rate_base = income.loc[mask_normal, "RoyaltyRate"].mean()
operating_cash_rate_base = income.loc[mask_normal, "OperatingCashRate"].mean()
other_operating_rate_base = operating_cash_rate_base - royalty_rate_base

# 2.3 Demand statistics (for Monte Carlo)
stud_std_mean = stud_hist["StudentWeeksStandard"].mean()
stud_std_std = stud_hist["StudentWeeksStandard"].std(ddof=1)
stud_wkd_mean = stud_hist["StudentWeeksWeekend"].mean()
stud_wkd_std = stud_hist["StudentWeeksWeekend"].std(ddof=1)

print("=== Parameters calibrated from CSVs ===")
print(f"Base standard price    : {price_std_base:,.0f} €/student-week")
print(f"Base weekend price     : {price_wkd_base:,.0f} €/weekend student")
print(f"Base royalty rate      : {royalty_rate_base:.3f}")
print(f"Other operating rate   : {other_operating_rate_base:.3f}")
print(f"Operating cash rate    : {operating_cash_rate_base:.3f}")
print(f"StdWeeks mean (hist)   : {stud_std_mean:.1f}  (std = {stud_std_std:.1f})")
print(f"WeekendStud mean (hist): {stud_wkd_mean:.1f}  (std = {stud_wkd_std:.1f})")

# Store in globalParams as defaults if needed
globalParams["price_std_base"] = float(price_std_base)
globalParams["price_wkd_base"] = float(price_wkd_base)
globalParams["royalty_rate_base"] = float(royalty_rate_base)
globalParams["other_operating_rate_base"] = float(other_operating_rate_base)

globalParams["stud_std_mean"] = float(stud_std_mean)
globalParams["stud_std_std"] = float(stud_std_std)
globalParams["stud_wkd_mean"] = float(stud_wkd_mean)
globalParams["stud_wkd_std"] = float(stud_wkd_std)


# Align base-case demand with the IESE case


# Historical means (already stored in globalParams)
mean_std = globalParams["stud_std_mean"]   # ~372 student-weeks (2017–2022)
mean_wkd = globalParams["stud_wkd_mean"]   # historical weekend average

# --- RELE base: plan 2022 ---
# They plan 320 standard weeks and 60 weekend students in 2022.
target_rele_std = 320.0
target_rele_wkd = 60.0

params["RELE"]["demand_mult_std"] = target_rele_std / mean_std
params["RELE"]["demand_mult_wkd"] = target_rele_wkd / mean_wkd

# --- OILTS base: more volume than RELE 2022, but not extreme ---
# For base case we assume ~550 standard weeks, 0 weekend for now.
target_oilts_std = 550.0
params["OILTS"]["demand_mult_std"] = target_oilts_std / mean_std
params["OILTS"]["demand_mult_wkd"] = 0.0  # explicitly no weekend program in base


# 3. Basic historical plots


# Plot 1: Total income vs profit after tax
plt.figure()
plt.plot(income["Year"], income["TotalIncome"], marker="o", label="Total income")
plt.plot(income["Year"], income["ProfitAfterTax"], marker="o", label="Profit after tax")
plt.title("Total income vs profit after tax (2017–2022)")
plt.xlabel("Year")
plt.ylabel("€")
plt.legend()
plt.tight_layout()
plt.show()

# Plot 2: Royalty and operating cash cost ratios
plt.figure()
plt.plot(income["Year"], income["RoyaltyRate"], marker="o", label="Royalty / income")
plt.plot(income["Year"], income["OperatingCashRate"], marker="o",
         label="Operating cash costs / income")
plt.title("Royalty and operating cash cost ratios")
plt.xlabel("Year")
plt.ylabel("Ratio")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 7. Run analysis


# Deterministic status quo for RELE (no Monte Carlo)
status_quo = analyze_status_quo(globalParams, params, strategy="RELE", plot=True)

# Full Monte Carlo for all strategies
horizons = [2, 5, 7, 10]
all_rows = []

for scen in scenario_defs:
    scen_name = scen["name"]
    print(f"\n=== Running scenario: {scen_name} ===")
    for T in horizons:
        gp_s, ps_s = apply_scenario(globalParams, params, scen)
        gp_s["T"] = T

        results_s, summary_s = run_all_strategies(gp_s, ps_s, strategies)

        for row in summary_s:
            row_out = row.copy()
            row_out["scenario"] = scen_name
            row_out["T"] = T
            all_rows.append(row_out)
            print(row_out)

summary_df = pd.DataFrame(all_rows)
print("\n=== Consolidated summary (head) ===")
print(summary_df.head())

#Donwload file
summary_df.to_csv("data/scenario_summary.csv", index=False)
summary_df.head()